In [1]:
import sys
!{sys.executable} -m pip install -qU langchain-huggingface


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [33]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [34]:

from langchain_ollama import ChatOllama
# Initialize your LLM
llm = ChatOllama(model="llama3.1")

In [35]:

from owlready2 import get_ontology, Thing, Restriction, And, Or, Not
from langchain_core.documents import Document

# Load your ontology
onto = get_ontology("./Ontology_Assignment.rdf").load()

#convert restriction into text
def restriction_to_text(restriction):

    #get name of property the restriction applies to
    #if property is None fallback to string, this shouldnt happen, but just incase 
    prop_name = restriction.property.name if restriction.property else str(restriction.property)

    # Check for "exists" attribute
    if hasattr(restriction, "some_values_from") and restriction.some_values_from: #check attribute and not empty
        #get name from target class or fallback to object itself
        target = getattr(restriction.some_values_from, "name", restriction.some_values_from)
        return f"must have some {prop_name} from {target}"
    
    # Check for "all" attribute
    elif hasattr(restriction, "all_values_from") and restriction.all_values_from:
        target = getattr(restriction.all_values_from, "name", restriction.all_values_from)
        return f"can only have {prop_name} from {target}"
    
    # Check for "≥" attribute
    elif hasattr(restriction, "min_cardinality") and restriction.min_cardinality is not None: #add is not None, because "0" would be handled as False
        return f"{prop_name} ≥ {restriction.min_cardinality}"
    
    # Check for "≤" attribute
    elif hasattr(restriction, "max_cardinality") and restriction.max_cardinality is not None: #add is not None, because "0" would be handled as False
        return f"{prop_name} ≤ {restriction.max_cardinality}"
    
    #Fallback for other types of restrictions
    else:
        return f"{prop_name} with unknown restriction {restriction}"

#recursive function that turns expression to text
def expression_to_text(expr):

    #Ontology class
    if hasattr(expr, "name"):
        return expr.name
    
    #constraint in ontology
    elif isinstance(expr, Restriction):
        return restriction_to_text(expr)
    
    #And expression
    elif isinstance(expr, And):
        return " and ".join(expression_to_text(c) for c in expr.Classes)
    
    #Or expression
    elif isinstance(expr, Or):
        return " or ".join(expression_to_text(c) for c in expr.Classes)
    
    #Not expr
    elif isinstance(expr, Not):
        return f"not {expression_to_text(expr.Class)}"

    #Fallback, return string
    else:
        return str(expr)

#turns ontology class to text
def class_to_text(cls):
    lines = [f"The class {cls.name} is defined as follows:"]
    
    # Superclasses
    for parent in cls.is_a:
        lines.append(f"- Subclass of {expression_to_text(parent)}")
    
    # Equivalent classes
    for eq in cls.equivalent_to:
        lines.append(f"- Equivalent to {expression_to_text(eq)}")
    
    return "\n".join(lines)

# turn object or data property to text
def property_to_text(prop):
    #starting text
    lines = [f"The property {prop.name} is defined as follows:"]
    
    #check for "domain" attribute and not empty
    if hasattr(prop, "domain") and prop.domain:
        #convert each class in domain to text
        domain_text = ", ".join(expression_to_text(d) for d in prop.domain)
        lines.append(f"- Domain: {domain_text}")
    
    #check for "range" attribute and not empty
    if hasattr(prop, "range") and prop.range:
        #convert each class in range to text
        range_text = ", ".join(expression_to_text(r) for r in prop.range)
        lines.append(f"- Range: {range_text}")
    
    # Only include characteristics if they exist
    if hasattr(prop, "is_functional") and prop.is_functional:
        lines.append("- This property is functional (at most one value).")
    if hasattr(prop, "is_inverse_functional") and prop.is_inverse_functional:
        lines.append("- This property is inverse functional (each value has at most one subject).")
    if hasattr(prop, "is_transitive") and prop.is_transitive:
        lines.append("- This property is transitive.")
    if hasattr(prop, "is_symmetric") and prop.is_symmetric:
        lines.append("- This property is symmetric.")
    if hasattr(prop, "is_asymmetric") and prop.is_asymmetric:
        lines.append("- This property is asymmetric.")
    if hasattr(prop, "is_reflexive") and prop.is_reflexive:
        lines.append("- This property is reflexive (every individual is related to itself).")
    if hasattr(prop, "is_irreflexive") and prop.is_irreflexive:
        lines.append("- This property is irreflexive (no individual is related to itself).")
    
    return "\n".join(lines)


# Collect all text for LLM
ontology_texts = []

# Add text for Classes
for cls in onto.classes():
    ontology_texts.append(class_to_text(cls))

# Add text for Object and datatype properties
for prop in onto.object_properties():
    ontology_texts.append(property_to_text(prop))
for prop in onto.data_properties():
    ontology_texts.append(property_to_text(prop))

# Turn into LLM documents
docs = []
for text in ontology_texts:
    prompt = f"Convert the following ontology snippet into clear, human-readable English. Do not add external information. Make sure the output only includes information from the ontology dont add any \"Here is the snippet\" type of text to the output \n\n{text}"
    human_text = llm.invoke(prompt)
    docs.append(Document(page_content=human_text.content))

# Add to your vector store
vector_store.add_documents(docs)


['06ceb0b0-a20e-4ce9-9e42-b55bfdcde3f0',
 '8eaab1e4-a75e-4968-86a2-49ac26ea4cbb',
 '98d05727-d489-4e0c-90fd-6ad26a4ddd5a',
 '5b019aae-9bbc-47f6-a7c1-2b1ea92d2adf',
 '88b2d430-9ff6-4f15-904d-4ca7b5ee786a',
 '31735d09-30d6-4a47-aa9e-9d903757c3a9',
 'c55d076c-2bac-4a0a-a918-eb579d336517',
 '8c71fcef-9bd8-4ed6-b696-a829720575bf',
 'e107cf87-5aed-4394-86dc-563c7e7d9052',
 'd1a7fd6f-7919-4d02-b0a6-bdf3f9163b22',
 '8183cd8e-fb99-4a3f-9f57-7c2cb8fc610e',
 '921360ab-b709-45e4-9a49-33db6b7a9950',
 '250cb8cf-589d-4a85-83eb-1f91eb0fc549',
 '74f016cb-e707-4d62-96ab-04f78515786f',
 'f95da1a2-5c98-4712-baaf-b0c8611ea2e0',
 'cb063f11-88c9-49bd-9c8c-0c9ebe9e0137',
 'e4aca343-3721-4f21-91b9-93a0223b8ba8',
 'dd7ce5e5-f74d-4d76-9baf-99d787fd883c',
 'fa006567-8e0e-44de-b746-fb23859f20e5',
 '2ca16fbb-7d1e-497f-bfcc-b959bb2dc03c',
 '92c56e90-cd59-4457-bd53-9dff73e55823',
 '60c55f49-3cb5-460f-be67-7c9dec1d3bfc',
 'aaba5d2e-5014-4b9f-a5da-9b18f06e0b6b',
 '9a3ee660-e184-4df8-9a10-159edc89ab8f',
 '87a6a93c-232f-

In [36]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [43]:
question = "Is Alex allowed to be married at 15?"
#question = "John and Amira are on vacation in Italy with the daughter Anna, Anna tells them she can't wait to visit the EifelTower. Do you see any inconsistencies in this story based on the context given, how would you change the story to make it consistent? Don't use any knowledge that isnt from the context"
k = 5
retrieved_docs = vector_store.similarity_search(question, k=k)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
rendered_prompt = prompt.invoke({"question": question, "context": docs_content})
answer = llm.invoke(rendered_prompt)

In [44]:
answer

AIMessage(content="No, Alex is not allowed to be married at 15 because the context specifies that the other individual must be an adult or elder, and Alex is considered a child of an adult. A person's marital status is based on their relationship with another individual, who must be an adult. Therefore, Alex cannot be married until they reach adulthood.", additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-10-16T15:59:16.0170567Z', 'done': True, 'done_reason': 'stop', 'total_duration': 931072900, 'load_duration': 80417100, 'prompt_eval_count': 138, 'prompt_eval_duration': 135390200, 'eval_count': 69, 'eval_duration': 678030100, 'model_name': 'llama3.1'}, id='run--60128c70-52f3-4794-b3bf-6cc73f83e50a-0', usage_metadata={'input_tokens': 138, 'output_tokens': 69, 'total_tokens': 207})

In [45]:
docs_content

'An individual can be married to another individual if that other individual is an adult or an elder.\n\nMarriage is a type of Life Event.\n\nA person can be a child of an adult.\n\nA person has a parent who is an adult.\n\nThe minimum required age for an occupation is an integer value.'